In [ ]:
!pip install ultralytics supervision==0.20.0 opencv-python deep-sort-realtime -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 126.3 MB/s eta 0:00:00


In [ ]:
import cv2
import supervision as sv
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort

# 🔹 Set your video path here
video_path = "/content/lost.mp4"   # <<--- change this to your video file path

# Load YOLO model + DeepSORT
model = YOLO("yolov8n.pt")
tracker = DeepSort(max_age=30)

# Trackers
bag_owner = {}
unattended_timer = {}
theft_alert = set()

def is_near(box1, box2, threshold=100):
    """Check if two boxes are near each other"""
    x1, y1, x2, y2 = box1
    cx1, cy1 = (x1+x2)//2, (y1+y2)//2
    x3, y3, x4, y4 = box2
    cx2, cy2 = (x3+x4)//2, (y3+y4)//2
    dist = ((cx1-cx2)**2 + (cy1-cy2)**2) ** 0.5
    return dist < threshold

# Open video
cap = cv2.VideoCapture(video_path)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter("output.mp4", fourcc, cap.get(cv2.CAP_PROP_FPS),
                      (int(cap.get(3)), int(cap.get(4))))

frame_count = 0
UNATTENDED_THRESHOLD = int(cap.get(cv2.CAP_PROP_FPS) * 5)  # 5 sec

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    frame_count += 1

    # Run YOLO
    results = model(frame, verbose=False)[0]
    detections = []

    for box in results.boxes:
        cls = int(box.cls)
        conf = float(box.conf)
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        if conf < 0.5:
            continue
        # Detect people + bags (backpack=24, handbag=25, suitcase=26)
        if cls in [0, 24, 25, 26]:
            detections.append([[x1, y1, x2-x1, y2-y1], conf, cls])

    # Update DeepSORT tracker
    tracks = tracker.update_tracks(detections, frame=frame)
    people, bags = {}, {}

    for t in tracks:
        if not t.is_confirmed():
            continue
        l, t_, r, b = map(int, t.to_ltrb())
        track_id = t.track_id
        cls = t.get_det_class()

        if cls == 0:
            people[track_id] = (l, t_, r, b)
            cv2.rectangle(frame, (l, t_), (r, b), (255, 255, 0), 2)
            cv2.putText(frame, f"Person {track_id}", (l, t_-10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,255,0), 2)
        elif cls in [24, 25, 26]:
            bags[track_id] = (l, t_, r, b)

    # Handle bag logic
    for bag_id, bag_box in bags.items():
        owner_id = None
        for pid, pbox in people.items():
            if is_near(bag_box, pbox, threshold=120):
                owner_id = pid
                break

        if owner_id is None:
            # Bag is unattended
            if bag_id not in unattended_timer:
                unattended_timer[bag_id] = frame_count
            if frame_count - unattended_timer[bag_id] > UNATTENDED_THRESHOLD:
                cv2.putText(frame, "⚠ Unattended Bag!", (bag_box[0], bag_box[1]-10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255), 2)
                cv2.rectangle(frame, (bag_box[0], bag_box[1]),
                              (bag_box[2], bag_box[3]), (0,0,255), 3)
        else:
            # Bag with an owner
            unattended_timer.pop(bag_id, None)
            if bag_id in bag_owner and bag_owner[bag_id] != owner_id:
                theft_alert.add(bag_id)
            bag_owner[bag_id] = owner_id

        # Theft detection persistent alert
        if bag_id in theft_alert:
            cv2.putText(frame, "🚨 Theft Detected!", (bag_box[0], bag_box[1]-10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255), 2)
            cv2.rectangle(frame, (bag_box[0], bag_box[1]),
                          (bag_box[2], bag_box[3]), (0,0,255), 3)

    out.write(frame)

cap.release()
out.release()

print("✅ Processing complete. Output saved as output.mp4")


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
✅ Processing complete. Output saved as output.mp4
